# Running SATURN

This notebook will demonstrate how to run SATURN and review the output files.

Make sure to run through `dataloader.ipynb` first.

First, make the run csv file.

It should have format:

|path|species|embedding path|
|-----|------|--------------|
|path to frog|frog|frog embeddings path|
|path to zebrafish|zebrafish|zebrafish embeddings path|

In [1]:
# Make the csv
import pandas as pd
import os

df = pd.DataFrame(columns=["path", "species", "embedding_path"])
df["species"] = ["frog", "zebrafish"]
df["path"] = ["data/frog.h5ad", "data/zebrafish.h5ad"]

##### CHANGE THESE PATHS #####
frog_embedding_path = os.path.join(os.environ['HOME'], 'protein_embeddings_export', 'ESM2', 'frog_embedding.torch')# "/dfs/project/cross-species/yanay/data/proteome/embeddings/Xenopus_tropicalis.Xenopus_tropicalis_v9.1.gene_symbol_to_embedding_ESM2.pt"
zebrafish_embedding_path = os.path.join(os.environ['HOME'], 'protein_embeddings_export', 'ESM2', 'zebrafish_embedding.torch')# "/dfs/project/cross-species/yanay/data/proteome/embeddings/Danio_rerio.GRCz11.gene_symbol_to_embedding_ESM2.pt"
##############################
df["embedding_path"] = [frog_embedding_path, zebrafish_embedding_path]
df.to_csv("data/frog_zebrafish_run.csv", index=False)
df

,path,species,embedding_path
0,data/frog.h5ad,frog,/home/tcroll/protein_embeddings_export/ESM2/fr...
1,data/zebrafish.h5ad,zebrafish,/home/tcroll/protein_embeddings_export/ESM2/ze...


# Scoring while training

We will score our model output while training. To do that, we will need a scoring csv file. We have provided one in this dataset at `data/frog_zebrafish_cell_type_map.csv`. It looks like this:

In [2]:
pd.read_csv("data/frog_zebrafish_cell_type_map.csv").head(10)

,Unnamed: 0,frog_cell_type,zebrafish_cell_type
0,0,Blastula,NaN
1,1,Germline,Germline
2,2,Neuroectoderm,Neuroectoderm
3,3,Non-neural ectoderm,Non-neural ectoderm
4,4,Involuting marginal zone,Involuting marginal zone
5,5,Spemann organizer,NaN
6,6,Endoderm,Endoderm
7,7,Epidermal progenitor,Epidermal progenitor
8,8,Ionocyte,Ionocyte
9,9,Goblet cell,NaN


# Train the Model

We can see `train-saturn.py`'s potential arguments with `--help`.

In [3]:
!/usr/conda/bin/python3 ../../train-saturn.py --help

Global seed set to 0
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
usage: train-saturn.py [-h] [--in_data IN_DATA] [--device DEVICE]
                       [--device_num DEVICE_NUM] [--time_stamp TIME_STAMP]
                       [--org ORG] [--log_dir LOG_DIR] [--work_dir WORK_DIR]
                       [--seed SEED] [--in_label_col IN_LABEL_COL]
                       [--ref_label_col REF_LABEL_COL]
                       [--tissue_subset TISSUE_SUBSET]
                       [--tissue_column TISSUE_COLUMN] [--hv_genes HV_GENES]
                       [--hv_span HV_SPAN] [--num_macrogenes NUM_MACROGENES]
                       [--centroids_init_path CENTROIDS_INIT_PATH]
                       [--embedding_model {ESM1b,MSA1b,protXL,ESM1b_protref,ESM2}]
                       [--vae [VAE]] [--hidden_dim HIDDEN_DIM]
                       [--model_dim MODEL_DIM]
                       [--binarize_expression [BINARIZE_EXPRESSION]]
        

We'll train SATURN with the following settings:

|Argument|Value|Explanation|
|--------|-----|-----------|
|in_data|`data/frog_zebrafish_run.csv`|The csv we created containing paths.|
|in_label_col|`cell_type`|Use the `cell_type` column labels for metric learning. **NOTE:** SATURN is weakly supervised, it does not share cell type labels across species, so you don't need to match these values across AnnDatas.|
|ref_label_col|`cell_type`|Extra cell type argument, will be added to our output but won't effect results since we didn't add `--use_ref_labels|
|num_macrogenes|`2000`|By default, we use 2000 macrogenes.|
|hv_genes|`8000`|By default, we use the 8000 most highly variable genes.|
|centroids_init_path|`saturn_results/fz_centroids.pkl`|Since this is the first time we are runinng this command, we will have to initialize our macrogenes using KMeans, which is an expensive operation. We save that initialization to this location so that if we pass this path to this argument in future runs, we can skip that step.|
|score_adata||By adding this flag, we will score our adatas after pretraining and while fine tuning with metric learning.|
|ct_map_path|`data/frog_zebrafish_cell_type_map.csv`|The path to our cell type mapping file, needed since we are scoring while training.|
|work_dir|`./`|SATURN outputs to a folder called `saturn_results`.|

SATURN is very verbose. Some things to check during model training:
- Do the AnnData views printed at the start have enough genes? These AnnData views are output after subsetting your input AnnDatas to just the genes that have protein embeddings.


**This command may take some time.**

GPU Memory usage: ~8GB for Pretraining, ~10GB total for metric learning but this may very.

In [ ]:
!/usr/conda/bin/python3 ../../train-saturn.py --in_data=data/frog_zebrafish_run.csv \
                              --in_label_col=cell_type --ref_label_col=cell_type \
                              --num_macrogenes=2000     --hv_genes=8000          \
                              --centroids_init_path=saturn_results/fz_centroids.pkl \
                              --score_adata --ct_map_path=data/frog_zebrafish_cell_type_map.csv \
                              --work_dir=. 

Global seed set to 0
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Using Device 0
Set seed to 0
After loading the anndata frog View of AnnData object with n_obs × n_vars = 96935 × 9538
    obs: 'clusters', 'parent_clusters', 'cell_type', 'n_genes', 'species', 'species_type_label', 'truth_labels', 'ref_labels'
    var: 'n_cells'
After loading the anndata zebrafish View of AnnData object with n_obs × n_vars = 63371 × 16980
    obs: 'n_counts', 'unique_cell_id', 'cell_names', 'library_id', 'batch', 'ClusterID', 'ClusterName', 'TissueID', 'TissueName', 'TimeID', 'cluster', 'cell_type', 'n_genes', 'species', 'species_type_label', 'truth_labels', 'ref_labels'
    var: 'n_cells'
Making Centroids


# Analyze SATURN Outputs

Let's check what files SATURN outputted:

In [5]:
!ls ./saturn_results

ls: cannot access './saturn_results': No such file or directory


We have a number of log files and 
- our output AnnData: `test256_data_frog_zebrafish_org_saturn_seed_0.h5ad`
- out ouput gene to macrogene weights: `test256_data_frog_zebrafish_org_saturn_seed_0_genes_to_macrogenes.pkl`

## Load SATURN Results

In [6]:
import scanpy as sc
import pickle

In [7]:
adata = sc.read("saturn_results/test256_data_frog_zebrafish_org_saturn_seed_0.h5ad")
adata

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'saturn_results/test256_data_frog_zebrafish_org_saturn_seed_0.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Visualize our integration

In [ ]:
sc.pp.pca(adata)

In [ ]:
sc.pl.pca(adata, color="species", title="Species")
sc.pl.pca(adata, color="labels2", title="Cell Type") # The original cell type names

In [ ]:
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color="species", title="Species")
sc.pl.umap(adata, color="labels2", title="Cell Type")

# Macrogene Differential Expression

With SATURN, we can perform differential expression on macrogenes rather than genes.

In [ ]:
with open("saturn_results/test256_data_frog_zebrafish_org_saturn_seed_0_genes_to_macrogenes.pkl", "rb") as f:
    macrogene_weights = pickle.load(f)

In [ ]:
# macrogene weights is a dictionary of (species_{gene name}) : [gene to macrogen weight](1x2000)
macrogene_weights

In [ ]:
# Create a copy of the adata with macrogenes as the X values
macrogene_adata = sc.AnnData(adata.obsm["macrogenes"])
macrogene_adata.obs = adata.obs

In [ ]:
sc.tl.rank_genes_groups(macrogene_adata, groupby="labels2", groups=["Ionocyte"], method="wilcoxon")

In [ ]:
sc.pl.rank_genes_groups_dotplot(macrogene_adata)

In [ ]:
ionocytes_de_df = sc.get.rank_genes_groups_df(macrogene_adata, group="Ionocyte").head(5)
ionocytes_de_df

## Investigate these macrogenes by their top ranked genes

In [ ]:
def get_scores(macrogene):
    '''
    Given the index of a macrogene, return the scores by gene for that centroid
    '''
    scores = {}
    for (gene), score in macrogene_weights.items():
        scores[gene] = score[int(macrogene)]
    return scores

In [ ]:
# Display the top macrogenes and their highest weights
# NOTE: this will be seed dependent.
for macrogene in ionocytes_de_df["names"]:
    print(f"Macrogene {macrogene}")
    display(pd.DataFrame(get_scores(macrogene).items(), columns=["gene", "weight"])\
            .sort_values("weight", ascending=False)\
            .head(10))